In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import joblib

path_mvi = "content/MICRODADOS_DE_MVI_JAN_2004_A_OUT_2025.xlsx"
path_brasileirao = "content/campeonato-brasileiro-full.csv"

print("Lendo bases de dados...")
df_mvi = pd.read_excel(path_mvi, sheet_name="Plan1")
df_brasileirao = pd.read_csv(path_brasileirao)
print("Bases carregadas com sucesso!")
print("-" * 60)



Lendo bases de dados...


FileNotFoundError: [Errno 2] No such file or directory: '/content/MICRODADOS_DE_MVI_JAN_2004_A_OUT_2025.xlsx'

In [ ]:
print(df_mvi)

                    MUNICIPIO     REGIAO_GEOGRAFICA       SEXO  \
0                    BEZERROS               AGRESTE  MASCULINO   
1                    BEZERROS               AGRESTE  MASCULINO   
2                BUENOS AIRES          ZONA DA MATA  MASCULINO   
3                  CAMARAGIBE  REGIÃO METROPOLITANA   FEMININO   
4                     CARUARU               AGRESTE  MASCULINO   
...                       ...                   ...        ...   
84977              FEIRA NOVA               AGRESTE   FEMININO   
84978              PARANATAMA               AGRESTE  MASCULINO   
84979                  RECIFE               CAPITAL  MASCULINO   
84980             SAO CAETANO               AGRESTE  MASCULINO   
84981  VITORIA DE SANTO ANTAO          ZONA DA MATA  MASCULINO   

      NATUREZA JURIDICA       DATA   ANO  IDADE  TOTAL DE VITIMAS  MES  
0             HOMICIDIO 2004-01-01  2004   22.0                 1    1  
1             HOMICIDIO 2004-01-01  2004    NaN              

In [ ]:
possiveis_datas = [c for c in df_mvi.columns if 'data' in c.lower()]
data_coluna = possiveis_datas[0] if possiveis_datas else "DATA_OCORRENCIA"
df_mvi[data_coluna] = pd.to_datetime(df_mvi[data_coluna], errors='coerce')
df_mvi = df_mvi.dropna(subset=[data_coluna])

possiveis_munic = [c for c in df_mvi.columns if 'mun' in c.lower()]
municipio_col = possiveis_munic[0] if possiveis_munic else "MUNICIPIO"

df_mvi["ANO"] = df_mvi[data_coluna].dt.year
df_mvi["MES"] = df_mvi[data_coluna].dt.month
vitimas_mes = df_mvi.groupby([municipio_col, "ANO", "MES"]).size().reset_index(name="TOTAL_VITIMAS")

print("Agrupamento de homicídios concluído:")
print(vitimas_mes.head())
print("-" * 60)

Agrupamento de homicídios concluído:
      MUNICIPIO   ANO  MES  TOTAL_VITIMAS
0  ABREU E LIMA  2004    1              4
1  ABREU E LIMA  2004    2              4
2  ABREU E LIMA  2004    3              6
3  ABREU E LIMA  2004    4              8
4  ABREU E LIMA  2004    5              5
------------------------------------------------------------


In [ ]:
df_brasileirao["data"] = pd.to_datetime(df_brasileirao["data"], errors="coerce")


df_brasileirao["ANO"] = df_brasileirao["data"].dt.year
df_brasileirao["MES"] = df_brasileirao["data"].dt.month
jogos_mes = df_brasileirao.groupby(["ANO", "MES"]).size().reset_index(name="TEM_JOGO")
jogos_mes["TEM_JOGO"] = 1

vitimas_mes = vitimas_mes.merge(jogos_mes, on=["ANO", "MES"], how="left")
vitimas_mes["TEM_JOGO"] = vitimas_mes["TEM_JOGO"].fillna(0).astype(int)

print("Variável TEM_JOGO adicionada com sucesso!")
print(vitimas_mes.head())
print("-" * 60)



Variável TEM_JOGO adicionada com sucesso!
      MUNICIPIO   ANO  MES  TOTAL_VITIMAS  TEM_JOGO
0  ABREU E LIMA  2004    1              4         0
1  ABREU E LIMA  2004    2              4         0
2  ABREU E LIMA  2004    3              6         0
3  ABREU E LIMA  2004    4              8         1
4  ABREU E LIMA  2004    5              5         1
------------------------------------------------------------


/tmp/ipython-input-3576602101.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_brasileirao["data"] = pd.to_datetime(df_brasileirao["data"], errors="coerce")


In [ ]:
le = LabelEncoder()
vitimas_mes["MUNICIPIO_ENCODED"] = le.fit_transform(vitimas_mes[municipio_col])

X = vitimas_mes[["MUNICIPIO_ENCODED", "ANO", "MES", "TEM_JOGO"]]
y = vitimas_mes["TOTAL_VITIMAS"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

#modelo um
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

#mdelo dois
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)




In [ ]:
def avaliar_modelo(nome, y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n {nome}")
    print(f"MAE  (Erro Médio Absoluto): {mae:.2f}")
    print(f"RMSE (Raiz do Erro Quadrático Médio): {rmse:.2f}")
    print(f"R²   (Coeficiente de Determinação): {r2:.3f}")
    return r2

r2_rf = avaliar_modelo("Random Forest", y_test, y_pred_rf)
r2_lr = avaliar_modelo("Regressão Linear", y_test, y_pred_lr)

print("\nComparativo final:")
melhor = "Random Forest" if r2_rf > r2_lr else "Regressão Linear"
print(f" Melhor modelo: {melhor}")
print("-" * 60)



 Random Forest
MAE  (Erro Médio Absoluto): 1.19
RMSE (Raiz do Erro Quadrático Médio): 2.07
R²   (Coeficiente de Determinação): 0.916

 Regressão Linear
MAE  (Erro Médio Absoluto): 3.18
RMSE (Raiz do Erro Quadrático Médio): 7.11
R²   (Coeficiente de Determinação): 0.002

Comparativo final:
 Melhor modelo: Random Forest
------------------------------------------------------------


In [ ]:
print(" Teste o modelo com seus próprios valores!")
ano = int(input("Digite o ANO (ex: 2026): "))
mes = int(input("Digite o MÊS (1-12): "))
municipio = input("Digite o MUNICÍPIO: ").upper()
tem_jogo = int(input("Digite 1 se houve jogo do Brasileirão neste mês, senão 0: "))

if municipio not in le.classes_:
    print("Município não encontrado no dataset.")
else:
    mun_encoded = le.transform([municipio])[0]
    X_novo = pd.DataFrame({
        "MUNICIPIO_ENCODED": [mun_encoded],
        "ANO": [ano],
        "MES": [mes],
        "TEM_JOGO": [tem_jogo]
    })

    pred_rf = rf.predict(X_novo)[0]
    pred_lr = lr.predict(X_novo)[0]

    print("\n Estimativas de vítimas:")
    print(f"Random Forest: {pred_rf:.1f}")
    print(f" Regressão Linear: {pred_lr:.1f}")
    print(f" Modelo mais confiável: {melhor}")


 Teste o modelo com seus próprios valores!
Digite o ANO (ex: 2026): 2024
Digite o MÊS (1-12): 5
Digite o MUNICÍPIO: recife
Digite 1 se houve jogo do Brasileirão neste mês, senão 0: 1

 Estimativas de vítimas:
Random Forest: 44.0
 Regressão Linear: 3.5
 Modelo mais confiável: Random Forest


In [ ]:
if r2_rf > r2_lr:
    melhor_modelo = rf
    nome_modelo = "modelo_random_forest.pkl"
else:
    melhor_modelo = lr
    nome_modelo = "modelo_regressao_linear.pkl"

joblib.dump(melhor_modelo, nome_modelo)

print(f"Modelo '{nome_modelo}' salvo com sucesso!")

💾 Modelo 'modelo_random_forest.pkl' salvo com sucesso!
